# Develop the final state class
## There are 3 parts of the class the user is primarily responsible for
1. Calculations specific to each topology e.g. start time, missing mass,....
2. Calculations common to all topologies, typically the Kinematics of the reaction
3. The Data to be saved in an output tree

## Topology Specific
This requires defining the Topo_X functions where C is the number of the topology.
These are saved in a seperate file ToposXXX.h. We will edit this directly in this Jupyter notebook.

Here you can set the reaction start time to allow calculation of DeltaTime for PID purposes. It may be you want to take this from one particular type of particle such as scattered electron or tagger photon.
You should also calculate the missing mass for this topology and other masses specific to a topology.

Again you will need to change XXX to your class name

**Note to save the file you must run the jupyter cell**

You will probably want to use the reconstructed missing 4-vector as the atualll missing particle 4-vector. To do this, after calculating HSLorentzVector miss give it to the missing particle in the topology and fix its mass to the PDGMass, you can use the particle FixP4 function for this. For example, if the only particle I did not detect for a particular topology was the proton I can calculate miss and:

fProton.FixP4(miss);

In [ ]:
%edit ToposPi2.h

In [1]:
%%file ToposPi2.h
#include "Pi2.h"
void Pi2::Topo_0(){
  //For topology Electron:Proton:Pip:Pim

  //if(fElectron.Detector()>0) {fGoodEvent=kFALSE;return;} //Put some cuts on particle detectors etc...
  //Define starttime from electron candidate
  fTrigger.StartTime(&fElectron);
  //Subtract starttime from all particles
  fTrigger.SubtractStartTime(&fElectron,&fProton,&fPip,&fPim);

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam + fTarget -fElectron.P4() -fProton.P4() -fPip.P4() -fPim.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
}
void Pi2::Topo_1(){
  //For topology Electron:Proton:Pip

  //if(fElectron.Detector()>0) {fGoodEvent=kFALSE;return;} //Put some cuts on particle detectors etc...
  //Define starttime from electron candidate
  fTrigger.StartTime(&fElectron);
  //Subtract starttime from all particles
  fTrigger.SubtractStartTime(&fElectron,&fProton,&fPip);

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam + fTarget -fElectron.P4() -fProton.P4() -fPip.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fPim.FixP4(miss);
}
void Pi2::Topo_2(){
  //For topology Electron:Proton:Pim

  //if(fElectron.Detector()>0) {fGoodEvent=kFALSE;return;} //Put some cuts on particle detectors etc...
  //Define starttime from electron candidate
  fTrigger.StartTime(&fElectron);
  //Subtract starttime from all particles
  fTrigger.SubtractStartTime(&fElectron,&fProton,&fPim);

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam + fTarget -fElectron.P4() -fProton.P4() -fPim.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fPip.FixP4(miss);
}
void Pi2::Topo_3(){
  //For topology Electron:Pip:Pim

  //if(fElectron.Detector()>0) {fGoodEvent=kFALSE;return;} //Put some cuts on particle detectors etc...
  //Define starttime from electron candidate
  fTrigger.StartTime(&fElectron);
  //Subtract starttime from all particles
  fTrigger.SubtractStartTime(&fElectron,&fPip,&fPim);

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam + fTarget -fElectron.P4() -fPip.P4() -fPim.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  
  fProton.FixP4(miss);
}


Created file '/work/Dropbox/HaSpect/dev/HASPECT6/hsclas12/ToposPi2.h'.


## Common Calculations : Kinematics
The final state kinematics are common to all topologies so we only need to write one function.
We can also make use of the Kinematics calculator HSKinematics (use as fKine) for calculating standard production and decay quantities. Remember to prefix any variable you wish to save with "TM." then add them to the TreeData class in the next step.

In [2]:
%edit KinematicsPi2.h

In [3]:
%%file KinematicsPi2.h
#include "Pi2.h"
void Pi2::Kinematics(){
  TD.TrigNSector=fTrigger.TrigNSectors();
  TD.TrigNSectorRoads=fTrigger.TrigNSectorsRoads();

   //Calculate possible resonances
  HSLorentzVector pMeson=fPip.P4()+fPim.P4();
  HSLorentzVector pDpp=fProton.P4()+fPip.P4();
  HSLorentzVector pD0=fProton.P4()+fPim.P4();
  //invariant masses
  TD.MesonMass=pMeson.M();
  TD.DppMass=pDpp.M();
  TD.D0Mass=pD0.M();
  
  fKine.SetElecsTarget(fBeam,fElectron.P4(),fTarget);
  TD.W=fKine.W(); //photon bem energy
  TD.Q2=fKine.Q2();
  TD.Pol=fKine.GammaPol();
  TD.Egamma=fKine.GammaE();
  
  //calculate CM production kinematics for meson
  fKine.SetMesonBaryon(pMeson,fProton.P4());
  fKine.ElectroCMDecay();
  TD.MesonCosTh=fKine.CosTheta();//prefix all variables to be saved wiht TM.
  TD.MesonPhi=fKine.Phi();
  
  TD.ElTh=fElectron.P4p()->Theta()*TMath::RadToDeg();
  TD.ElP=fElectron.P4p()->P();
  TD.ElTime=fElectron.DeltaTime();
  TD.ProtTh=fProton.P4p()->Theta()*TMath::RadToDeg();
  TD.ProtP=fProton.P4p()->P();
  TD.ProtTime=fProton.DeltaTime();
  TD.PipTh=fPip.P4p()->Theta()*TMath::RadToDeg();
  TD.PipP=fPip.P4p()->P();
  TD.PipTime=fPip.DeltaTime();
  TD.PimTh=fPim.P4p()->Theta()*TMath::RadToDeg();
  TD.PimP=fPim.P4p()->P();
  TD.PimTime=fPim.DeltaTime();

  
}

Created file '/work/Dropbox/HaSpect/dev/HASPECT6/hsclas12/KinematicsPi2.h'.


## Tree Data
The data you write in a tree is connected via a TreeData class which has data members which will be the tree leafs i.e. data you can perform further analysis on (e.g. histogram, event fitting). Every variable you add to the class can be accessed in your final state class with TM.var=some number;

In [ ]:
%edit TreeDataPi2.h

In [2]:
%%file TreeDataPi2.h
#ifndef TREEDATAPi2_h
#define TREEDATAPi2_h 
#include "TreeData.h"

class TreeDataPi2: public HS::TreeData{
 public:
  //data member for tree branches below here
  Double_t MissMass=0;
  Double_t MissMass2=0;
  
  Short_t TrigNSector=0;
  Short_t TrigNSectorRoads=0;

  Double_t MesonMass=0;
  Double_t DppMass=0;
  Double_t D0Mass=0;
  
  Double_t W=0; 
  Double_t Q2=0;
  Double_t Pol=0;
  Double_t Egamma=0;
  
  Double_t MesonCosTh=0;
  Double_t MesonPhi=0;  
  
  Double_t ElTh=0;
  Double_t ElP=0;
  Double_t ElTime=0;
  Double_t ProtTh=0;
  Double_t ProtP=0;
  Double_t ProtTime=0;
  Double_t PipTh=0;
  Double_t PipP=0;
  Double_t PipTime=0;
  Double_t PimTh=0;
  Double_t PimP=0;
  Double_t PimTime=0;


  ClassDef(TreeDataPi2,1);
};
#endif


Created file '/scratch/dglazier/testg8/HASPECT6/hsclas12/TreeDataPi2.h'.


**Compiling**
Now you can try and compile all of the code together. You may need to go back and fix bugs after. This is best done by removing the %edit cells and doing a restart and run whole notebook. Ore restarting and clicking through all cells apart from %edit

In [3]:
gROOT->ProcessLine(".x LoadPi2.C+");

%%%%%%%%%%%%%%%%%%%%%%%%%    THSParticle


In file included from LoadFinalState_C_ACLiC_dict dictionary payload:10:
/scratch/dglazier/testg8/HASPECT6/hsfinalstate/LoadFinalState.C:11:6: error: redefinition of 'LoadFinalState'
void LoadFinalState(){
     ^
/scratch/dglazier/tesths/HASPECT6/hsfinalstate/LoadFinalState.C:11:6: note: previous definition is here
void LoadFinalState(){
     ^
Warning in <TInterpreter::TCling::RegisterModule>: Problems declaring payload for module LoadFinalState_C_ACLiC_dict.


%%%%%%%%%%%%%%%%%%%%%%%%%    EventInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    RunInfo
%%%%%%%%%%%%%%%%%%%%%%%%%    THSWeights
%%%%%%%%%%%%%%%%%%%%%%%%%    DataManager
%%%%%%%%%%%%%%%%%%%%%%%%%    LundReader
&&&&&&&&&&&&&&&&&&&&&&&&&&&& HSKinematics
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Cuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Combitorial
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleIter
&&&&&&&&&&&&&&&&&&&&&&&&&&&& Topology
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FinalState
&&&&&&&&&&&&&&&&&&&&&&&&&&&& FiledTree
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreeData
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TopoActionManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCuts
&&&&&&&&&&&&&&&&&&&&&&&&&&&& VarsParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepParticle
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalID
&&&&&&&&&&&&&&&&&&&&&&&&&&&& ParticleCutsManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& TreePrepManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& MVASignalIDManager
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS/CLASTrigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS12/CLAS12Trigger
&&&&&&&&&&&&&&&&&&&&&&&&&&&& CLAS

Info in <TUnixSystem::ACLiC>: creating shared library /scratch/dglazier/testg8/HASPECT6/hsclas12/./TreeDataPi2_C.so
Info in <TUnixSystem::ACLiC>: creating shared library /scratch/dglazier/testg8/HASPECT6/hsclas12/./Pi2_C.so


Now you can try running on some data, here is an example of how to run on a [ROOT file](RunROOT.ipynb)